# Bus Access and Income in Los Angeles: Final Notebook 2
**_Arturo Jacobo & Miranda Mead-Newton_**

For our final research project, we are exploring bus access and income in Los Angeles County, and have made recommendations according to our findings. Bus riders in Los Angeles are some of the most low-income populations in the City and changes are coming to the bus network. We sought to examine whether these planned changes are going to improve bus network coverage for low income populations. We found that very little coverage changes are planned for the new NextGen bus network. We examine the areas of the City we believe deserve higher coverage.

In this notebook, we dive into the changes in the new planned bus network: NextGen and develop a metric for bus access for the County. We then look at the spatial relationship between income and bus access through an interactive Kepler map. 

In [ ]:
# libraries
import urllib.request, json 
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from keplergl import KeplerGl
from sodapy import Socrata

# new for data viz
import seaborn as sns

# to explore point patterns
from pointpats import centrography
from matplotlib.patches import Ellipse
import numpy

## NextGen and the existing bus network
First, we wanted to compare the changes in NextGen and the existing bus network. We did this by plotting the two networks on top of each other. 

In [ ]:
NextGen = gpd.read_file('data/Next_Gen/200820_NextGen_Transit_First_DX15.shp')
ExRoute = gpd.read_file('data/Existing_Bus_System/Existing_Bus_System.shp')

NextGen needed to be reprojected in order for them to be mapped one on top of the other. 

In [ ]:
NextGen.crs

In [ ]:
NextGen_web = NextGen.to_crs('epsg:3857')

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
ax.axis('off')
ExRoute.plot(ax=ax)
NextGen_web.plot(ax=ax,color='g')
ctx.add_basemap(ax)

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
ax.axis('off')

NextGen_web.plot(ax=ax,color='g')
ExRoute.plot(ax=ax)
ctx.add_basemap(ax)

Above is a comparison of the NextGen bus network and the existing bus network. The maps show that though the NextGen plan is a thorough one, it will not dramatically change the physical bus network. The biggest changes in the plan are condensing several bus lines and increasing service. And we don't have access to frequency data. 

Below are some detailed shots of areas of LA that we eventually compared to our map of bus access. 

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
ax.axis('off')
ExRoute.plot(ax=ax)
NextGen_web.plot(ax=ax,color='g')


ax.set_xlim(-1.319e7, -1.315e7)
ax.set_ylim(4.017e6, 4.04e6)
ctx.add_basemap(ax)
ax.set_title("Downtown and South LA")

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))
ax.axis('off')
ExRoute.plot(ax=ax)
NextGen_web.plot(ax=ax,color='g')

ax.set_xlim(-1.322e7, -1.313e7)
ax.set_ylim(4.027e6, 4.08e6)
ctx.add_basemap(ax)
ax.set_title("San Fernando Valley and North of Downtown")

## Measuring Bus Access

### Bus Stops and Bus Stop Distribution in LA

In our analysis of bus access in LA, we began by looking at the concentration of bus stops in the County. The map below uses a dataset of LA Metro bus stops for each bus line. If a bus stop serves multiple buses, it is weighted as such. This is a rough metric for access to the bus across the County. This map shows the median and mean center of bus stops and shows their distribution across a vertical and horizontal access. Using this map, we can see that bus stops are more heavily concentrated in the center of the City. The dip in the vertical distribution shows the lack of bus access in the hills between the valley and the center of the city. We sharpened our bus access metric by taking into account bus frequency in the next step.

In [ ]:
lss = gpd.read_file('data/Bus_Stops/JUN20_Lines_Serving_Stops.shp')

In [ ]:
lss_web_mercator = lss.to_crs(epsg=3857)

In [ ]:
# compute the mean and median centers
mean_center = centrography.mean_center(lss[['LONG','LAT']])
med_center = centrography.euclidean_median(lss[['LONG','LAT']])

In [ ]:
# Generate scatter plot
g = sns.jointplot(
    x='LONG', y='LAT', data=lss, s=3, 
)
g.fig.set_figwidth(15)
g.fig.set_figheight(10)

# Add mean point and marginal lines
g.ax_joint.scatter(
    *mean_center, color='red', marker='x', s=50, label='Mean Center'
)
g.ax_marg_x.axvline(mean_center[0], color='red')
g.ax_marg_y.axhline(mean_center[1], color='red')

# Add median point and marginal lines
g.ax_joint.scatter(
    *med_center, color='limegreen', marker='o', s=50, label='Median Center'
)
g.ax_marg_x.axvline(med_center[0], color='limegreen')
g.ax_marg_y.axhline(med_center[1], color='limegreen')

# Legend
g.ax_joint.legend()

# Add basemap
ctx.add_basemap(
    g.ax_joint, 
    crs='epsg:4326',
    source=ctx.providers.CartoDB.Positron
)

# Clean axes
g.ax_joint.set_axis_off()

# Display
plt.show()

### Creating the Bus Access Metric

One of the biggest challenges we ran into was developing a metric for bus access. We had quite a few limitations when using the data from Metro including a lack of information on the datasets and a lack of current data. The metric we settled on for bus access used bus stop information and headway data. We started with a dataset from Metro called Lines Serving Stops from June 2020. This dataset includes the geographic location of every bus stop in the existing LA Metro network data organized by bus line. This means that if a bus stop is served by four bus lines it will appear four times in the dataset. (Note that the description on the website for the Lines Serving Stops and Stops Serving Lines datasets are reversed). Just mapping this dataset gives a fairly good representation of bus access (more bus stops mean more bus lines and therefore greater access to the City).

Next, we appended the bus frequency data from the headways dataset (note that this data is from 2014 and discrepancies between 2014 and the above 2020 data resulted in some dropped bus stops). We totaled the number of buses that run on each line in a week during peak hours by summing the “total trips” by bus line and multiplying the weekday trips by five. Next, we assigned each bus stop with the total weekly trips that pass through it. Each of these weekly trips can be seen as an “opportunity” to take the bus. In a sense, we are measuring bus access by the number of buses that one has the chance to take during a week. This does not take into account where one might be able to take the bus, accessibility of various services or employment via the bus, bus wait time, bus reliability, or walking distance to the bus.

Below is that process:

In [ ]:
#import headways data (downloaded from metro as an excel file, resaved a .csv)
hw = gpd.read_file('data/JUN14HeadwayLineDirDaytypeTimeperiod.csv')

In [ ]:
# here's what the dataset looks like
hw.head(30)

The challenge here is to sum total trips by line while taking into account the fact that these trips are divided into 3 different "Day Types." The day types are: 1 (weekdays), 2 (saturdays), 3 (sundays). We want the total trips for saturdays and sundays but also add weekdays but weight them by a factor of 5 in order to get the week total. 

In [ ]:
# Turn the trips data into an integer so it can be added
hw['Trips'] = hw['Trips'].astype(int)

First we can use `groupby` to get trip totals for the day types (which adds up time of day totals and directional totals). 

In [ ]:
hwtt = hw.groupby(["Line","DayType"]).Trips.sum().reset_index()
hwtt

In [ ]:
# This will create a list of every bus line which we will use later
buslines = hwtt.Line.unique().tolist()

In [ ]:
# creating a dataframe based on the list of bus lines
bt = pd.DataFrame(data=buslines)
bt

In [ ]:
# renaming the single column
bt.columns = ['Line']

Using our hwtt list with trip information, we're going to create a new column on the `bt` dataframe with total trips. Weekdays are multiplied by 5 to create a weighted week total.

In [ ]:
for x in buslines:
    a = hwtt.loc[(hwtt['Line'] == x) & ((hwtt['DayType'] == '2') | (hwtt['DayType'] == '3')),'Trips'].sum()
    b = hwtt.loc[((hwtt['Line'] == x) & (hwtt['DayType'] == '1')),'Trips'].sum()
    bt.loc[(bt['Line'] == x),'TotalTrips'] = a + b*5
bt

Next, in order to add the week total for each bus stop according to its line, we will create a dictionary out of the `bt` dataframe in order to reference it. Note that I'm sure there's an easier way to do all this, but this ended up working out!

In [ ]:
btd = dict(zip(bt.Line, bt.TotalTrips))
btd

In [ ]:
# The lines in the bus line dictionary are strings, so we'll have to make the bus lines in lss also strings
lss['LINE'] = lss['LINE'].astype(str)

In [ ]:
# this creates a new column with total weekly trips using the stop's bus line and the dictionary for reference
lss['WeeklyTrips'] = lss['LINE'].map(btd)

In [ ]:
# now we have a bus access metric that we can map in the next step
lss.head()

### Mapping Bus Access and Income

The map below superimposes our Bus Access metric over a map of income for the County of LA. Lower income areas are blue and higher income areas are white. Higher levels of bus service are yellow and lower levels of bus service are purple. The areas of interest in this map are those where incomes are low and bus access is also low. Low income populations tend to be much more reliant on the bus and are less likely to own a car. We were curious to compare these areas of interest with the new NextGen bus plan.

In [ ]:
inc = gpd.read_file('data/acs2019_5yr_B19013_14000US06037432102.geojson')

In [ ]:
inc = inc.drop([0])

In [ ]:
inc.columns = ['geoid',
'name',
'Income',
'Error',
'geometry']

In [ ]:
map = KeplerGl(height=600,width=800)
map

In [ ]:
map.add_data(data=lss,name='Bus Access')

In [ ]:
map.add_data(data=inc,name='Income_County')

Within Kepler, we added two layers to the map. The first layer was our bus access metric. We used hexabins with a radius of 0.1 and a height scale based on "WeeklyTrips." We used the fourth color option and made use to base Color on "WeeklyTrips" as well. 

The second layer we used Income_County as the datasource and used Polygons to visualize the census income data. We used "Income" for the color visualization and picked reversed white to teal as the colors. We played around with the polygon opacity so that the information was clear but the labels from the map were still visible. We then saved the map and uploaded it to github (From Miranda: I did this in a weird way were I accidentally created a new repository, so I'm not sure if I'm able to share the file very easily but it can be viewed at the website below or on our story map). Note that the file saved with the name below to this jupyterhubs folder is not the final draft and is just a test from running this notebook. 

https://mmeadnewton.github.io/busstopsincome.html

In [ ]:
# map.save_to_html(file_name='busstopsincome.html',read_only=True)

## ^the code above is turned into a note so that we don't save over the current file. Normally, it would be used
## to save our map file. 

## Conclusion
One area on this map of note is the Valley/North Hollywood. Though the Valley is served by the Orange and Red Lines, there are few options for crossing into Central LA and this commute can become very congested. We were interested in the fact that North Hollywood is characterized by lower incomes and low bus access. On the map on the right, you can see that some of the existing network will be removed in this area.

Parts of South and East LA are also characterized by lower incomes. These areas have higher frequencies of bus service, however, most of the transit access in the area runs into Downtown LA. There is less neighborhood travel coverage in these areas. Though these bus lines run regularly, they are more spaced apart south of the 10 freeway. This area will also see some reduction in bus line coverage.

Without being able to analyze frequency changes, it's clear that there are current areas characterized by low incomes and low level of bus service. From our research and findings, we know that low income populations tend to be most dependent on the bus and have less car access. These areas tend to be further away from the central city and likely commute into Central LA. Increasing service in these areas would be an easy way to reduce overall vehicle miles traveled (VMT). This is probably especially true in the Valley where there are few routes into the City. More bus access could greatly reduce peak rush hour congestion.

In South LA, we noticed that most of the current service appears to serve those who want to travel to Downtown and Central LA. Most of the high level frequency bus routes travel North/South and those routes are closer together leading to greater coverage. This is useful for job access but may leave gaps for those who wish to travel within their neighborhood or run errands. This can further isolate those living in poverty from community and basic resources. There's also no easy route for those in South LA to travel to West LA or Century City, another job center.

In East LA, there are areas characterized by lower incomes and a very sparse bus network. Similar, to South LA, travel is mostly aimed at commuters into downtown and there's very little opportunity for intra-community travel.

See our storymaps for more information:
https://storymaps.arcgis.com/stories/c97e33fc08ca44c59df8736261558f74